In [1]:
!uv pip install seaborn

Resolved 17 packages in 232ms                                        
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)--------------     0 B/288.00 KiB          
⠙ Preparing packages... (0/1)-------------- 16.00 KiB/288.00 KiB        
⠙ Preparing packages... (0/1)-------------- 32.00 KiB/288.00 KiB        
⠙ Preparing packages... (0/1)-------------- 48.00 KiB/288.00 KiB        
⠙ Preparing packages... (0/1)-------------- 64.00 KiB/288.00 KiB        
⠙ Preparing packages... (0/1)-------------- 80.00 KiB/288.00 KiB        
⠙ Preparing packages... (0/1)-------------- 96.00 KiB/288.00 KiB        
⠙ Preparing packages... (0/1)-------------- 112.00 KiB/288.00 KiB       
⠙ Preparing packages... (0/1)-------------- 128.00 KiB/288.00 KiB       
⠙ Preparing packages... (0/1)-------------- 144.00 KiB/288.00 KiB       
⠙ Preparing packages... (0/1)m------------- 160.00 KiB/288.00 KiB       
⠙ Preparing packages... (0/1)----------- 176.0

In [2]:
import math
import numpy as np
import seaborn as sns
import matplotlib as plt
from tqdm import tqdm
import torch.nn as nn

In [4]:
import torch
import torch.nn as nn

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 20),
            nn.Tanh(),
            nn.Linear(20, 30),
            nn.Tanh(),
            nn.Linear(30, 30),
            nn.Tanh(),
            nn.Linear(30, 20),
            nn.Tanh(),
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 1)
        )

    def forward(self, x):
        out = self.net(x)
        return out


In [ ]:
class Net:
    def __init__(self, x):
        
        if torch.cuda.is_available():
            device = torch.device("cuda")
        elif torch.backends.mps.is_available():
            device = torch.device("mps")
        else:
            device = torch.device("cpu")

        self.model = NN().to(device)

        # comp. domain
        self.h = 0.1
        self.k = 0.1
        x = torch.arrange(-1,1+self.h, self.h)
        t = torch.arrange(0,1+self.k, self.k)

        self.X = torch.stack(torch.meshgrid(x,t)).reshape(2,-1).T

        # train data

        bc1 = torch.stack(torch.meshgrid(x[0],t)).reshape(2,-1).T
        bc2 = torch.stack(torch.meshgrid(x[-1],t)).reshape(2,-1).T
        ic = torch.stack(torch.meshgrid(x,t[0])).reshape(2,-1).T
        self.x_train = torch.cat([bc1, bc2, ic])

        y_bc1 = torch.zeros(len(bc1))
        y_bc2 = torch.zeros(len(bc2))          
        y_ic = -torch.sin(math.pi* ic[:,0])
        self.y_train=torch.cat([y_bc1,y_bc2,y_ic])
        self.y_train=self.y_train.unsqueeze(1)

        self.x = self.X.to(device)
        self.y_train = self.y_train.to(device)
        self.x_train = self.x_train.to(device)
        self.x.requires_grad = True
        self.adam = torch.optim.Adam(self.model.parameters())
        self.optimizer = torch.optim.LBFGS(
            self.model.parameters(),
            lr=1.0,
            max_iter=50000,
            max_eval=50000,
            history_size=50,
            tolerance_grad=1e-7,
            tolerance_change=1.0*np.finfo(float).eps,
            line_search_fn="strong_wolfe"
        )
        self.criterion = torch.nn.MSELoss()
        self.iter = 1

    def loss_func(self):
        self.adam.zero_grad()
        self.optimizer.zero_grad()

        y_pred = self.model(self.x_train)
        loss_data = self.criterion(y_pred, self.y_train)